In [6]:
from predict_label import Predictor
import numpy as np
import glob

In [7]:
#!/usr/bin/env python
import numpy as np
import pickle 
from keras.models import load_model
from keras.preprocessing.image import  img_to_array, load_img
import face_recognition as fr
from PIL import Image

class Predictor:
    def __init__(self, kerasmodelfile):
        self.kerasmodelfile = kerasmodelfile
        self.picklefile = kerasmodelfile.replace("h5", "pickle")
        self.model = load_model(self.kerasmodelfile)
        with open(self.picklefile, 'rb') as handle:
            self.ids_to_classes = pickle.load(handle)
    
    def convert_to_face_only(self, ifile, final_height=120, final_width=120, raw_resize_fraction=0.5):
        image = fr.load_image_file(ifile)
        image = np.rot90(image,-1)
        face_locations = fr.face_locations(image)
        top,right,bottom,left = face_locations[0]
        face_image = image[top:bottom, left:right]
        small_img = Image.fromarray(face_image)
        small_img = small_img.resize((final_height, final_width))
        return np.copy(np.array(small_img)[:,:,:3])

    def predict(self, imagefn):
        x = self.convert_to_face_only(imagefn)
        x = x.astype('float32')
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

        x /= 255.
        prob = self.model.predict(x)
        #print(prob,np.argmax(prob),len(ids_to_classes))
        return self.ids_to_classes[np.argmax(prob)]
#def predict(imagefn, kerasmodelfile, picklefile):
#    model = load_model(kerasmodelfile)
#    with open(picklefile, 'rb') as handle:
#        ids_to_classes = pickle.load(handle)
#    img = load_img(imagefn)  # this is a PIL image    
#    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
#    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
#    prob = model.predict(x)
#    print(prob,np.argmax(prob),len(ids_to_classes))



In [8]:
pred = Predictor("try_run_people_model.h5")

In [9]:
for f in glob.glob("img/Andreas_Steffen/*.jpg"):
    print(f,pred.predict(f))

img/Andreas_Steffen/5c0d794c-7981-46cf-a838-70d32f92b255.jpg Andreas_Steffen
img/Andreas_Steffen/6af3854c-b617-4341-b7a9-c2daee4b4c2b.jpg Andreas_Steffen
img/Andreas_Steffen/6ead320d-f835-4b14-a9c5-ecbde445042c.jpg Andreas_Steffen
img/Andreas_Steffen/a017f937-bd63-4785-bcc4-37b989ea3af7.jpg Andreas_Steffen
img/Andreas_Steffen/bcc03a0f-09a4-4f64-bad7-717c1d64b0e0.jpg Andreas_Steffen
img/Andreas_Steffen/e474894c-9087-4a4d-93d6-cbead6a629f5.jpg Andreas_Steffen
img/Andreas_Steffen/e802dd21-0ad4-4c65-84f9-2e0c498e8644.jpg Andreas_Steffen
img/Andreas_Steffen/f7fa35d1-3e34-4966-8e2c-396ace8f8bfb.jpg Andreas_Steffen
